In [3]:
import sqlite3
import pandas as pd  # Optional, for better data display

In [1]:
def read_sqlite_database(db_path):
    """
    Read and explore SQLite database content
    """
    try:
        # Connect to database
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # Get all table names
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        
        print(f"Tables in database: {[table[0] for table in tables]}")
        
        # For each table, show structure and sample data
        for table in tables:
            table_name = table[0]
            print(f"\n--- Table: {table_name} ---")
            
            # Get table schema
            cursor.execute(f"PRAGMA table_info({table_name});")
            columns = cursor.fetchall()
            print("Columns:")
            for col in columns:
                print(f"  {col[1]} ({col[2]})")
            
            # Get sample data (first 5 rows)
            cursor.execute(f"SELECT * FROM {table_name} LIMIT 5;")
            rows = cursor.fetchall()
            print(f"\nSample data (first 5 rows):")
            for row in rows:
                print(f"  {row}")
        
        conn.close()
        
    except sqlite3.Error as e:
        print(f"Database error: {e}")
    except Exception as e:
        print(f"Error: {e}")

def query_specific_data(db_path, query):
    """
    Execute a specific SQL query
    """
    try:
        conn = sqlite3.connect(db_path)
        
        # Using pandas for better display (optional)
        if 'pandas' in globals():
            df = pd.read_sql_query(query, conn)
            print(df)
        else:
            cursor = conn.cursor()
            cursor.execute(query)
            rows = cursor.fetchall()
            for row in rows:
                print(row)
        
        conn.close()
        
    except sqlite3.Error as e:
        print(f"Database error: {e}")
    except Exception as e:
        print(f"Error: {e}")



In [4]:
# Example usage
if __name__ == "__main__":
    db_path = "kite_app.db"  # Replace with your database path
    
    # Explore the database
    read_sqlite_database(db_path)

Tables in database: ['api_credentials', 'sqlite_sequence']

--- Table: api_credentials ---
Columns:
  id (INTEGER)
  api_key (TEXT)
  api_secret (TEXT)
  access_token (TEXT)
  created_at (TIMESTAMP)

Sample data (first 5 rows):
  (1, 'van071gzk4gqyzxz', '1lk6hx7aq1k243anwjtb8jee3qxlrzt9', 'BoWvBXTJPbQJVrQxcneMOIm27PYcUUTn', '2025-08-02 08:22:54')

--- Table: sqlite_sequence ---
Columns:
  name ()
  seq ()

Sample data (first 5 rows):
  ('api_credentials', 1)


In [ ]:
# KiteTickerWrapper Usage in Jupyter Notebook
# Import the KiteTickerWrapper class from the python file

import time
import pandas as pd
from datetime import datetime
from IPython.display import display, clear_output

# Import KiteTickerWrapper from the python file
from kite_websocket import KiteTickerWrapper

# Configuration
API_KEY = "your_api_key_here"
ACCESS_TOKEN = "your_access_token_here"

# Sample instrument tokens (replace with your desired instruments)
INSTRUMENTS = {
    'RELIANCE': 738561,    # Reliance Industries
    'TCS': 2953217,        # Tata Consultancy Services  
    'INFY': 408065,        # Infosys
    'HDFC': 340481,        # HDFC Bank
    'ICICIBANK': 1270529   # ICICI Bank
}

# Data storage for notebook
tick_data = []
live_prices = {}

# Initialize KiteTickerWrapper directly
ticker = None

def initialize_ticker(api_key, access_token):
    """Initialize KiteTickerWrapper with callbacks"""
    global ticker
    ticker = KiteTickerWrapper(api_key, access_token, debug=True)
    setup_callbacks()
    return ticker
def setup_callbacks():
    """Setup all the callback functions for KiteTickerWrapper"""
    global ticker
    
    def on_connect(ticker_instance):
        print(f"✅ Connected to Kite Ticker at {datetime.now()}")
        
    def on_disconnect(ticker_instance):
        print(f"❌ Disconnected from Kite Ticker at {datetime.now()}")
        
    def on_error(error):
        print(f"🚨 Ticker Error: {error}")
        
    def on_tick(ticker_instance, ticks):
        """Process incoming tick data"""
        global tick_data, live_prices
        
        for tick in ticks:
            # Store tick data
            tick_data.append({
                'timestamp': datetime.now(),
                'instrument_token': tick['instrument_token'],
                'last_price': tick['last_price'],
                'volume': tick.get('volume', 0),
                'change': tick.get('change', 0)
            })
            
            # Update live prices
            token = tick['instrument_token']
            live_prices[token] = {
                'price': tick['last_price'],
                'volume': tick.get('volume', 0),
                'timestamp': datetime.now(),
                'change': tick.get('change', 0)
            }
        
        # Keep only last 1000 ticks to avoid memory issues
        if len(tick_data) > 1000:
            tick_data = tick_data[-1000:]
            
        print(f"📊 Received {len(ticks)} ticks at {datetime.now().strftime('%H:%M:%S')}")
    
    # Assign callbacks to ticker
    ticker.on_connect(on_connect)
    ticker.on_disconnect(on_disconnect) 
    ticker.on_error(on_error)
    ticker.on_message(on_tick)
        
def start_ticker():
    """Start the ticker connection"""
    global ticker
    print("🚀 Starting Kite Ticker...")
    success = ticker.connect(threaded=True)
    if success:
        print("✅ Ticker started successfully")
    else:
        print("❌ Failed to start ticker")
    return success
    
def stop_ticker():
    """Stop the ticker connection"""
    global ticker
    print("🛑 Stopping Kite Ticker...")
    ticker.disconnect()
    print("✅ Ticker stopped")
    
def subscribe_instruments(instruments, mode="ltp"):
    """Subscribe to instruments"""
    global ticker
    tokens = list(instruments.values()) if isinstance(instruments, dict) else instruments
    ticker.subscribe(tokens, mode)
    
    instrument_names = list(instruments.keys()) if isinstance(instruments, dict) else tokens
    print(f"📈 Subscribed to {len(tokens)} instruments: {instrument_names}")
    
def unsubscribe_instruments(instruments):
    """Unsubscribe from instruments"""
    global ticker
    tokens = list(instruments.values()) if isinstance(instruments, dict) else instruments
    ticker.unsubscribe(tokens)
    print(f"📉 Unsubscribed from {len(tokens)} instruments")
    
def get_live_data_df():
    """Get current live data as DataFrame"""
    if not live_prices:
        return pd.DataFrame()
        
    data = []
    for token, info in live_prices.items():
        # Find instrument name
        name = "Unknown"
        for inst_name, inst_token in INSTRUMENTS.items():
            if inst_token == token:
                name = inst_name
                break
                
        data.append({
            'Instrument': name,
            'Token': token,
            'Price': info['price'],
            'Volume': info['volume'],
            'Change': info['change'],
            'Last Update': info['timestamp'].strftime('%H:%M:%S')
        })
        
    return pd.DataFrame(data)
    
def get_tick_history_df(limit=100):
    """Get tick history as DataFrame"""
    if not tick_data:
        return pd.DataFrame()
        
    recent_ticks = tick_data[-limit:] if len(tick_data) > limit else tick_data
    df = pd.DataFrame(recent_ticks)
    
    # Add instrument names
    def get_instrument_name(token):
        for name, inst_token in INSTRUMENTS.items():
            if inst_token == token:
                return name
        return f"Token_{token}"
        
    df['instrument_name'] = df['instrument_token'].apply(get_instrument_name)
    return df[['timestamp', 'instrument_name', 'instrument_token', 'last_price', 'volume', 'change']]


# Initialize the ticker - you need to call this first
print("KiteTickerWrapper imported successfully!")
print("Available instruments:", INSTRUMENTS)
print("\nSetup steps:")
print("1. Run: ticker = initialize_ticker(API_KEY, ACCESS_TOKEN)")
print("2. Run: start_ticker()")
print("3. Run: subscribe_instruments(INSTRUMENTS, 'ltp')")  
print("4. Run: display_live_data() to see live updates")
print("5. Run: stop_ticker() when done")


# Utility functions for the notebook

def display_live_data():
    """Display live data in a formatted table"""
    df = get_live_data_df()
    if not df.empty:
        display(df.style.highlight_max(subset=['Price'], color='lightgreen')
                      .highlight_min(subset=['Price'], color='lightcoral')
                      .format({'Price': '₹{:.2f}', 'Volume': '{:,}', 'Change': '{:.2f}'}))
    else:
        print("No live data available yet...")

def display_tick_history(limit=20):
    """Display recent tick history"""
    df = get_tick_history_df(limit)
    if not df.empty:
        display(df.tail(limit).style.format({
            'last_price': '₹{:.2f}', 
            'volume': '{:,}',
            'change': '{:.2f}'
        }))
    else:
        print("No tick history available yet...")

def monitor_live_data(duration=30, update_interval=5):
    """Monitor live data for specified duration"""
    print(f"📊 Monitoring live data for {duration} seconds (updates every {update_interval}s)")
    print("Press Ctrl+C to stop monitoring\n")
    
    start_time = time.time()
    try:
        while time.time() - start_time < duration:
            clear_output(wait=True)
            print(f"🕐 Time elapsed: {int(time.time() - start_time)}s / {duration}s")
            print(f"📈 Live Market Data (Last updated: {datetime.now().strftime('%H:%M:%S')})")
            print("=" * 80)
            display_live_data()
            time.sleep(update_interval)
    except KeyboardInterrupt:
        print("\n⏹️ Monitoring stopped by user")

def get_price_summary():
    """Get a summary of current prices"""
    df = get_live_data_df()
    if df.empty:
        print("No data available")
        return
        
    print("📊 Price Summary:")
    print("=" * 40)
    for _, row in df.iterrows():
        print(f"{row['Instrument']:12} | ₹{row['Price']:8.2f} | Vol: {row['Volume']:,}")

# Example usage patterns for notebook cells:

def example_basic_usage():
    """Example of basic usage - run this in separate cells"""
    
    print("""
    # Cell 1: Initialize ticker
    ticker = initialize_ticker(API_KEY, ACCESS_TOKEN)
    
    # Cell 2: Start ticker and subscribe
    start_ticker()
    time.sleep(2)  # Wait for connection
    subscribe_instruments(INSTRUMENTS, 'ltp')
    
    # Cell 3: Display live data (run multiple times to see updates)
    display_live_data()
    
    # Cell 4: Monitor data continuously  
    monitor_live_data(duration=60, update_interval=10)
    
    # Cell 5: Check tick history
    display_tick_history(limit=10)
    
    # Cell 6: Get price summary
    get_price_summary()
    
    # Cell 7: Stop ticker when done
    stop_ticker()
    """)

def example_advanced_usage():
    """Example of advanced usage with custom analysis"""
    
    print("""
    # Advanced Usage Examples:
    
    # 1. Subscribe to different modes
    subscribe_instruments([738561], 'full')  # Full data for Reliance
    subscribe_instruments([408065, 2953217], 'quote')  # Quote for Infy, TCS
    
    # 2. Custom analysis of tick data
    df = get_tick_history_df(100)
    price_changes = df.groupby('instrument_name')['last_price'].agg(['min', 'max', 'mean'])
    print(price_changes)
    
    # 3. Real-time alerts (implement your logic)
    def check_price_alerts():
        df = get_live_data_df()
        for _, row in df.iterrows():
            if row['Change'] > 2:  # Alert if change > 2%
                print(f"🚨 Alert: {row['Instrument']} moved {row['Change']:.2f}%")
    
    # 4. Export data
    df = get_tick_history_df()
    df.to_csv(f'tick_data_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv')
    
    # 5. Get current subscriptions
    subscriptions = ticker.get_subscriptions()
    print("Current subscriptions:", subscriptions)
    """)

# Display usage examples
print("\n" + "="*60)
print("USAGE EXAMPLES:")
print("="*60)
example_basic_usage()